In [1]:
#Agregamos las bibliotecas que vayamos a usar y se crea un cliente
from google.cloud import bigquery
client = bigquery.Client()

Se va a tratar con los datos

In [2]:
#Se escogieron estos datos por
#
#
#
#
#Se agrega la referencia al conjunto de datos y se recogen
dataset_ref = client.dataset("covid19_vaccination_access", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)


In [3]:
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

facility_boundary_us_all
facility_boundary_us_drive
facility_boundary_us_transit
facility_boundary_us_walk


In [4]:
#Seleccionamos la tabla a ocupar
table_ref = dataset_ref.table("facility_boundary_us_all")
table = client.get_table(table_ref)

In [5]:
client.list_rows(table, max_results=5).to_dataframe()

,facility_place_id,facility_provider_id,facility_name,facility_latitude,facility_longitude,facility_country_region,facility_country_region_code,facility_sub_region_1,facility_sub_region_1_code,facility_sub_region_2,facility_sub_region_2_code,facility_region_place_id,mode_of_transportation,travel_time_threshold_minutes,facility_catchment_boundary
0,ChIJAQAAANfpFogRjmEdiZ7FYFE,aef36226-56fd-4243-aa31-958d9e702722,Martin's Supermarkets Pharmacy,41.6396,-85.9286,United States,US,Indiana,US-IN,Elkhart County,18039,ChIJHWkKPnXrFogR3u0vklO67mc,transit,60,"POLYGON((-85.842865 41.6103765, -85.896587 41...."
1,ChIJ4Y9VFvBgBYgRDRMMPxpzW6M,1277e03c-7b14-4a6c-8658-4c0615b2d960,Kenosha Pediatrics-Children's Wisconsin,42.5739,-87.9541,United States,US,Wisconsin,US-WI,Kenosha County,55059,ChIJ2Si-7u9QBYgRFx8FDcBlxJA,transit,60,"MULTIPOLYGON(((-87.927352 42.5916891, -87.9622..."
2,ChIJK1RrBW2F7ocRGb3B2PCDy24,71946341-0eed-48e4-898b-20ff15628bdf,CVS Pharmacy,41.7323,-93.5995,United States,US,Iowa,US-IA,Polk County,19153,ChIJu8uF2zgl7IcR6oIsamzvHr4,transit,60,"MULTIPOLYGON(((-93.6376137 41.711331, -93.6376..."
3,ChIJfU-rMMmr-IgRQg3m2N7c3hE,114a46d0-32bd-4f14-934f-737dd392b4b3,Kroger Pharmacy,34.1146,-80.8825,United States,US,South Carolina,US-SC,Richland County,45079,ChIJOxu60MdK_4gRQqZb8Meu5LU,transit,60,"MULTIPOLYGON(((-80.9546385 34.0262023, -80.954..."
4,ChIJm5ZVcXG3hVQRN6CqAW29gjw,None,Rite Aid,48.9347,-122.4850,United States,US,Washington,US-WA,Whatcom County,53073,ChIJi6KetAsXhVQRq7HXbC1bHz8,transit,60,"POLYGON((-122.540532 48.7968822, -122.526872 4..."


In [6]:
#Realicemos un query para tomar las columnas que se ven mas relevantes
#evitando columnas
#
query = """
                SELECT  facility_name, facility_country_region, facility_sub_region_1, mode_of_transportation, travel_time_threshold_minutes
                FROM `bigquery-public-data.covid19_vaccination_access.facility_boundary_us_all` 
         
        
        """

query_job = client.query(query)
Dato = query_job.to_dataframe()
Dato

,facility_name,facility_country_region,facility_sub_region_1,mode_of_transportation,travel_time_threshold_minutes
0,Martin's Supermarkets Pharmacy,United States,Indiana,transit,60
1,Kenosha Pediatrics-Children's Wisconsin,United States,Wisconsin,transit,60
2,CVS Pharmacy,United States,Iowa,transit,60
3,Kroger Pharmacy,United States,South Carolina,transit,60
4,Rite Aid,United States,Washington,transit,60
...,...,...,...,...,...
580645,Ambrose Clement Health Center,United States,Ohio,transit,60
580646,Lark Medical Associates,United States,Tennessee,transit,60
580647,Fry's Food And Drug,United States,Arizona,transit,60
580648,Walgreens Pharmacy,United States,Illinois,transit,60


In [8]:
#Contamos cuantas instalaciones posee cada pais
#
#
Dato.facility_country_region.value_counts()

facility_country_region
United States          578450
Puerto Rico              2090
U.S. Virgin Islands       110
Name: count, dtype: int64

In [9]:
#Realicemos un query para reducir aun mas los datos
#
#
query = """
                SELECT  facility_name, facility_sub_region_1, mode_of_transportation, travel_time_threshold_minutes
                FROM `bigquery-public-data.covid19_vaccination_access.facility_boundary_us_all` 
                WHERE facility_country_region = "United States";
        """

query_job = client.query(query)
Dato = query_job.to_dataframe()
Dato

,facility_name,facility_sub_region_1,mode_of_transportation,travel_time_threshold_minutes
0,Martin's Supermarkets Pharmacy,Indiana,transit,60
1,Kenosha Pediatrics-Children's Wisconsin,Wisconsin,transit,60
2,CVS Pharmacy,Iowa,transit,60
3,Kroger Pharmacy,South Carolina,transit,60
4,Rite Aid,Washington,transit,60
...,...,...,...,...
578445,Ambrose Clement Health Center,Ohio,transit,60
578446,Lark Medical Associates,Tennessee,transit,60
578447,Fry's Food And Drug,Arizona,transit,60
578448,Walgreens Pharmacy,Illinois,transit,60


In [10]:
#contemos los datos
#
#
col = Dato.keys()
for name in col:
    print(Dato[name].value_counts())

facility_name
CVS Pharmacy                          96350
Walgreens Pharmacy                    81150
Walmart Pharmacy                      41050
Rite Aid Pharmacy                     22720
Rite Aid                              12600
                                      ...  
Merriman Drugs                           10
Newton Family and Pediatric Clinic       10
Central Drug Store                       10
Ken's Thriftee Pharmacy                  10
Lorain Family Health Center              10
Name: count, Length: 19876, dtype: int64
facility_sub_region_1
California              64170
Florida                 43180
Texas                   38710
New York                29990
Ohio                    29830
Pennsylvania            28320
Georgia                 20760
Michigan                20070
Illinois                19730
North Carolina          19380
New Jersey              15660
Virginia                14590
Tennessee               14220
Louisiana               12720
Washington         

In [11]:
#Dato.mode_of_transportation.value_counts()
#Dato.travel_time_threshold_minutes.value_counts()
